In [6]:
import pandas as pd
import pickle
from scipy.spatial.distance import cdist
import numpy as np

In [3]:

class ClusterClassifier:
    def __init__(self):
        self.df = pd.read_parquet("df_final.parquet")

    def classify_embedding(self, embedding: list[float]) -> int:
        # columns: [["cluster_label", "etiqueta", "centroid"]]
        distances = cdist([embedding], self.df.centroid.tolist())
        return self.df.iloc[np.argmin(distances)]["cluster_label"]

In [ ]:
df = pd.read_parquet("../df_final.parquet")
df

In [ ]:

distances = cdist([df.centroid.iloc[0]], df.centroid.tolist())
distances


In [ ]:
df.iloc[np.argmin(distances)]["cluster_label"]


In [ ]:
import pandas as pd

# Cargar el DataFrame
df_final = pd.read_parquet("../df_final.parquet")
df_final["etiqueta"] = "minuscula"

# Comprobar la capitalización
print("Primeras 5 etiquetas originales:")
print(df_final["etiqueta"].head())

print("\nPrimeras 5 etiquetas capitalizadas:")
print(df_final["etiqueta"].str.capitalize().head())

# Verificar si todas las etiquetas están capitalizadas
todas_capitalizadas = df_final["etiqueta"].str[0].str.isupper().all()
print(f"\n¿Todas las etiquetas están capitalizadas? {todas_capitalizadas}")

# Mostrar etiquetas no capitalizadas (si las hay)
if not todas_capitalizadas:
    print("\nEtiquetas no capitalizadas:")
    print(df_final[~df_final["etiqueta"].str[0].str.isupper()]["etiqueta"])

In [40]:
df["vector_distance"] = df["centroid"].apply(lambda x: np.linalg.norm(x))

In [ ]:
# Normalizar los vectores
df["centroid_normalized"] = df["centroid"].apply(lambda x: x / np.linalg.norm(x))

# Calcular la nueva distancia del vector normalizado
df["vector_distance_normalized"] = df["centroid_normalized"].apply(lambda x: np.linalg.norm(x))

# Mostrar el DataFrame actualizado
print(df[["cluster_label", "etiqueta", "centroid_normalized", "vector_distance_normalized"]])

# Verificar que todas las distancias normalizadas sean aproximadamente 1
print("\n¿Todas las distancias normalizadas son aproximadamente 1?")
print(np.allclose(df["vector_distance_normalized"], 1.0, atol=1e-6))


In [ ]:

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient


client = SearchClient(endpoint, index_name, AzureKeyCredential(key))
results = list(client.search(search_text="*"))

In [4]:
import json

# with open("documentos_pre.json", "w") as f: 
#     json.dump(results, f)

### Coger docs de PRE

In [7]:
import json
results = json.load(open("documentos_pre.json", "r"))

In [8]:
results[0].keys()

dict_keys(['document_path', 'ts_file_update', 'category', 'description', 'original_canonic_path', 'content_vector', 'dt_end', 'document_id', 'file_size', 'calendar_name', 'location', 'site_name', 'sp', 'title', 'sheet_name', 'event_id', 'event_name', 'dt_start', 'content', 'etiqueta_cluster', 'event_url', 'ts_creation', 'original_path', 'content_md', 'document_name', 'chunk_id', 'content_type', 'id', 'ts_processed', 'page_number', '@search.score', '@search.reranker_score', '@search.highlights', '@search.captions'])

In [9]:
df = pd.DataFrame(results)[["id", "content", "content_vector"]]
df.head()

,id,content,content_vector
0,ayudadch_copilot_manuales_de_formacion_superme...,La tabla presenta una lista de productos relac...,"[-0.029008377, 0.019819578, -0.0022607665, 0.0..."
1,ayudadch_copilot_manuales_de_formacion_superme...,La tabla presenta un horario de actividades o ...,"[-0.023789804, 0.015766677, 0.0006295868, 0.02..."
2,supermercadodch_departamento_formacion_gestion...,No hay casos abiertos por el mismo motivo Inic...,"[-0.009540582, 0.03877473, -0.001934909, 0.023..."
3,supermercadodch_departamento_formacion_gestion...,La tabla presenta un inventario de productos c...,"[-0.016271949, 0.03131432, 0.00031013912, 0.03..."
4,ayudadch_copilot_manuales_de_formacion_superme...,Captura pedidos GPER DVD División de venta a d...,"[-0.002767793, 0.031104341, -0.0032851375, 0.0..."


In [4]:
from collections import defaultdict
from time import time

from sklearn import metrics

evaluations = []
evaluations_std = []


def fit_and_evaluate(km, X, name=None, n_runs=5):
    name = km.__class__.__name__ if name is None else name

    train_times = []
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        # scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        # scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        # scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        # scores["Adjusted Rand-Index"].append(
        #     metrics.adjusted_rand_score(labels, km.labels_)
        # )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

    return evaluations, evaluations_std

### Normalizar

In [10]:
# Normalizar los vectores en content_vector

# Función para normalizar un vector
def normalizar_vector(vector):
    norma = np.linalg.norm(vector)
    return vector / norma if norma != 0 else vector

# Aplicar la normalización a cada vector en la columna content_vector
df['content_vector_norm'] = df['content_vector'].apply(normalizar_vector)

print("Vectores normalizados con éxito.")


Vectores normalizados con éxito.


## Evaluar k

In [6]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score


def cluster_kmeans(embeddings, k, n_init=5, max_iter=500):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=n_init, max_iter=max_iter)
    cluster_labels = kmeans.fit_predict(embeddings)
    inertia = kmeans.inertia_  # Inercia
    silhouette_avg = silhouette_score(embeddings, cluster_labels)  # Silhouette Score
    return cluster_labels, inertia, silhouette_avg, kmeans.cluster_centers_

In [ ]:
N_INIT = 25
MAX_N_CLUSTERS = 100

from tqdm import tqdm


silhouettes = list()
inertias = list()
n_clusters = range(2, MAX_N_CLUSTERS)

for n in tqdm(n_clusters):
    final_cluster_labels, inertia, silhouette_avg, cluster_centers = cluster_kmeans(df["content_vector_norm"].tolist(), n, n_init=N_INIT)
    silhouettes.append(silhouette_avg)
    inertias.append(inertia)

# Plot the silhouettes
plt.plot(n_clusters, silhouettes, marker="o")
plt.xlabel("Número de clusters")
plt.ylabel("Coeficiente de silueta")
plt.title("Coeficiente de silueta para diferentes números de clusters")
plt.show()

# Plot the inertias
plt.plot(n_clusters, inertias, marker="o")
plt.xlabel("Número de clusters")
plt.ylabel("Inercia")
plt.title("Inercia para diferentes números de clusters")
plt.show()

In [ ]:
evaluations


In [ ]:
# Plot the silhouettes

fig, ax = plt.subplots(figsize = (10, 5))
ax.plot(n_clusters, silhouettes, marker="o")
ax.set_xlabel("Número de clusters")
plt.ylabel("Coeficiente de silueta")
plt.title("Coeficiente de silueta para diferentes números de clusters")
ax.grid(True)

plt.show()

In [ ]:
# Graficar la inercia

fig, ax = plt.subplots(figsize = (10, 5))
ax.plot(n_clusters, inertias, marker="o")
ax.set_xlabel("Número de clusters")
ax.set_ylabel("Inercia")
ax.set_title("Inercia para diferentes números de clusters")
ax.grid(True)

plt.show()


In [ ]:
best_k = 37

final_cluster_labels, inertia, silhouette_avg, cluster_centers = cluster_kmeans(df["content_vector_norm"].tolist(), best_k, n_init=50, max_iter=500)
df["cluster_label"] = final_cluster_labels
df.head()


In [ ]:
print("Silhouette Score: ", silhouette_avg, "Inertia: ", inertia)

In [ ]:
# Crear un gráfico de barras para visualizar el conteo de etiquetas de cluster
plt.figure(figsize=(12, 6))
df.cluster_label.value_counts().plot(kind='bar')
plt.title('Distribución de Etiquetas de Cluster')
plt.xlabel('Etiqueta de Cluster')
plt.ylabel('Conteo')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Visualización de clusters usando PCA de dos dimensiones

# Importar las bibliotecas necesarias
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.colors as mcolors

# Crear un objeto PCA con 2 componentes
pca = PCA(n_components=2)

# Ajustar y transformar los datos
pca_result = pca.fit_transform(df["content_vector_norm"].tolist())

# Crear un DataFrame con los resultados del PCA
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])
pca_df['cluster'] = df['cluster_label']

# Crear una paleta de colores altamente diferenciables
colores = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.CSS4_COLORS.values())
n_clusters = len(pca_df['cluster'].unique())
paleta_colores = colores[:n_clusters]

# Crear un gráfico de dispersión
plt.figure(figsize=(12, 8))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['cluster'], cmap=mcolors.ListedColormap(paleta_colores), alpha=0.7)

plt.title('Visualización de Clusters usando PCA')
plt.xlabel('Primera Componente Principal')
plt.ylabel('Segunda Componente Principal')

# Añadir una barra de color
plt.colorbar(scatter, label='Etiqueta de Cluster', ticks=range(n_clusters))

plt.tight_layout()
plt.show()

# Imprimir la varianza explicada por cada componente
print(f"Varianza explicada por la PC1: {pca.explained_variance_ratio_[0]:.2f}")
print(f"Varianza explicada por la PC2: {pca.explained_variance_ratio_[1]:.2f}")


In [ ]:
# Visualización interactiva de clusters usando PCA de tres dimensiones con Plotly

# Importar las bibliotecas necesarias
from sklearn.decomposition import PCA
import numpy as np
import plotly.express as px
import pandas as pd
import matplotlib.colors as mcolors

# Crear un objeto PCA con 3 componentes
pca_3d = PCA(n_components=3)

# Ajustar y transformar los datos
pca_result_3d = pca_3d.fit_transform(df["content_vector"].tolist())

# Crear un DataFrame con los resultados del PCA
pca_df_3d = pd.DataFrame(data=pca_result_3d, columns=['PC1', 'PC2', 'PC3'])
pca_df_3d['cluster'] = df['cluster_label']

# Definir la misma paleta de colores que en el gráfico anterior
colores = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.CSS4_COLORS.values())
n_clusters = len(pca_df_3d['cluster'].unique())
paleta_colores = colores[:n_clusters]

# Crear un diccionario de mapeo de cluster a color
color_map = {str(cluster): color for cluster, color in zip(pca_df_3d['cluster'].unique(), paleta_colores)}

# Crear un gráfico de dispersión 3D interactivo con Plotly
fig = px.scatter_3d(pca_df_3d, x='PC1', y='PC2', z='PC3', 
                    color='cluster', 
                    color_discrete_map=color_map,
                    title='Visualización Interactiva de Clusters usando PCA en 3D',
                    labels={'PC1': 'Primera Componente Principal',
                            'PC2': 'Segunda Componente Principal',
                            'PC3': 'Tercera Componente Principal'},
                    hover_data=['cluster'])

# Ajustar el diseño del gráfico y reducir el tamaño de los puntos
fig.update_layout(scene = dict(
                    xaxis_title='Primera Componente Principal',
                    yaxis_title='Segunda Componente Principal',
                    zaxis_title='Tercera Componente Principal'),
                  width=900, height=700,
                  margin=dict(r=20, b=10, l=10, t=40))

fig.update_traces(marker=dict(size=3))  # Reducir el tamaño de los puntos

# Mostrar el gráfico interactivo
fig.show()

# Imprimir la varianza explicada por cada componente
print(f"Varianza explicada por la PC1: {pca_3d.explained_variance_ratio_[0]:.2f}")
print(f"Varianza explicada por la PC2: {pca_3d.explained_variance_ratio_[1]:.2f}")
print(f"Varianza explicada por la PC3: {pca_3d.explained_variance_ratio_[2]:.2f}")


### Distribución distancias al centroide

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
import numpy as np
import matplotlib.pyplot as plt

def plot_histograms_and_identify_outliers(cluster_data, cluster_label):
    # Calculate the centroid
    centroid = np.mean(cluster_data, axis=0)
    distances = cosine_distances(cluster_data, [centroid]).flatten()
    
    # Calculate the threshold for identifying outliers
    threshold = np.percentile(distances, 95)  # Change the percentile if necessary
    outliers = distances[distances > threshold]

    # Plot the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(distances, bins=30, alpha=0.7, color='blue')
    plt.title(f'Histograma de distancias coseno al centroide - Cluster {cluster_label}')
    plt.xlabel('Distancia Coseno')
    plt.ylabel('Frecuencia')
    plt.grid()
    plt.axvline(x=np.mean(distances), color='red', linestyle='dashed', linewidth=1, label='Media')
    plt.axvline(x=threshold, color='green', linestyle='dashed', linewidth=1, label='Umbral de Outliers')
    plt.legend()
    plt.show()
    
    # Print the number of outliers and total embeddings
    total_embeddings = len(cluster_data)
    per_outliers= (len(outliers))*100/total_embeddings
    per_outliers = round(per_outliers, 3)
    print(f'Cluster {cluster_label}: {len(outliers)} outliers identificados de {total_embeddings} embeddings totales.')
    print(f'Percentage of total outliers is {per_outliers}%.')

# Generate histograms and count outliers for each cluster
for label in df['cluster_label'].unique():
    cluster_data = np.array(df[df['cluster_label'] == label]['content_vector_norm'].tolist())
    plot_histograms_and_identify_outliers(cluster_data, label)

print("Histogramas de distancias coseno al centroide generados y outliers identificados.")


## UMAP 

In [ ]:
# Importar las bibliotecas necesarias
import umap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Crear un objeto UMAP con 2 componentes
umap_model = umap.UMAP(n_components=2)

# Ajustar y transformar los datos
umap_result = umap_model.fit_transform(np.array(df["content_vector_norm"].tolist()))

# Crear un DataFrame con los resultados de UMAP
umap_df = pd.DataFrame(data=umap_result, columns=['UMAP1', 'UMAP2'])
umap_df['cluster'] = df['cluster_label']

# Crear una paleta de colores altamente diferenciables
colores = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.CSS4_COLORS.values())
n_clusters = len(umap_df['cluster'].unique())
paleta_colores = colores[:n_clusters]

# Crear un gráfico de dispersión
plt.figure(figsize=(12, 8))
scatter = plt.scatter(umap_df['UMAP1'], umap_df['UMAP2'], c=umap_df['cluster'], cmap=mcolors.ListedColormap(paleta_colores), alpha=0.7)

plt.title('Visualización de Clusters usando UMAP')
plt.xlabel('Primera Componente UMAP')
plt.ylabel('Segunda Componente UMAP')

# Añadir una barra de color
plt.colorbar(scatter, label='Etiqueta de Cluster', ticks=range(n_clusters))

plt.tight_layout()
plt.show()


In [ ]:
import umap
import pandas as pd
import numpy as np
import plotly.express as px

# Crear un objeto UMAP con 3 componentes
umap_model = umap.UMAP(n_components=3)

# Ajustar y transformar los datos
umap_result = umap_model.fit_transform(np.array(df["content_vector_norm"].tolist()))

# Crear un DataFrame con los resultados de UMAP
umap_df = pd.DataFrame(data=umap_result, columns=['UMAP1', 'UMAP2', 'UMAP3'])
umap_df['cluster'] = df['cluster_label'].astype(str)  # Convertir a string para la visualización
umap_df['id'] = df['id']

# Crear una visualización 3D interactiva con Plotly
fig = px.scatter_3d(umap_df, x='UMAP1', y='UMAP2', z='UMAP3', color='cluster',
                    title='Visualización Interactiva de Clusters usando UMAP en 3D',
                    labels={'UMAP1': 'Primera Componente UMAP',
                            'UMAP2': 'Segunda Componente UMAP',
                            'UMAP3': 'Tercera Componente UMAP'},
                             text='id',  # Usar el id como etiqueta
                    color_continuous_scale=px.colors.qualitative.Set1,
                    width=1000,  # Ancho de la figura
                    height=800)  # Alto de la figura

# Mostrar la figura
fig.show()


In [ ]:
df.head()

In [ ]:
import tiktoken

In [ ]:
def contar_tokens(texto, modelo="gpt-3.5-turbo"):
    # Cargar el codificador específico para el modelo
    codificador = tiktoken.encoding_for_model(modelo)
    # Convertir el texto en tokens
    tokens = codificador.encode(texto)
    # Contar la cantidad de tokens
    cantidad_tokens = len(tokens)
    return cantidad_tokens

# Obtener todos los documentos de la base de datos
def get_all_docs(search_client):
    results = search_client.search(search_text="*", top=None, include_total_count=True)
    documents = [result for result in results]
    return documents

# Obtener todos los embeddings sin agrupar por document_id
def get_all_embeddings(search_client):
    results = get_all_docs(search_client)
    print(f"Embeddings tratados: {len(results)}")
    embeddings = []
    document_ids = []  # Para mantener una referencia a los IDs de los documentos
    document_paths = []  # Para mantener los document_path
    contents = []  # Para mantener el contenido de cada documento
    total_documents = len(results)

    for doc in results:
        document_id = doc.get('id')  # ID del documento (nuevo campo)
        document_path = doc.get('document_path')  # Path del documento
        embedding = doc.get('content_vector')  # Vector de embedding
        content = doc.get('content')  # Contenido del documento

        if embedding is not None:
            embeddings.append(embedding)  # Añadir el embedding directamente a la lista
            document_ids.append(document_id)  # Mantener un registro del document_id
            document_paths.append(document_path)  # Mantener un registro del document_path
            contents.append(content)  # Almacenar el contenido también

    print(f'Número total de documentos en el estudio de cluster: {total_documents} documentos')
    print(f'Número total de embeddings procesados: {len(embeddings)}')

    return np.array(embeddings), document_ids, document_paths, contents  # Devolver también el contenido


# Función para generar etiquetas llamando al modelo de GPT
def generate_cluster_label(contents):
    prompt = ("Analiza el texto que te voy a proporcionar a continuación y genera dos o varias etiquetas que represente el contenido con un máximo de 5."
        "Quiero que la etiqueta sea con una dos o tres palabras para dar detalle, y que clasifique el contenido por temas. "
        "No incluyas la palabra 'Etiqueta' en la respuesta y asegúrate de que no esté en formato de lista. "
        " Etiqueta esta en lenguaje español.\n\n"
    )
    combined_contents = "\n".join(contents)  # Añadir los contenidos
    total_tokens = contar_tokens(prompt) + contar_tokens(combined_contents)

    if total_tokens > 125000:
        # Si excede el límite, truncar o dividir los contenidos
        combined_contents = " ".join(contents)[:125000 - contar_tokens(prompt)]  # Truncar para ajustarse
        contents = [combined_contents]  # Usar el contenido truncado

    prompt += combined_contents  # Añadir el contenido truncado

    # Llamar a la función call_chatgpt_model para obtener la etiqueta
    etiqueta = call_chatgpt_model(prompt, model, assistant)
    return etiqueta

In [ ]:

# Procesar los embeddings
embeddings, document_ids, document_paths, contents = get_all_embeddings(search_client)  # Obtener también contenidos

# Realizar el clustering final usando el mejor k
final_cluster_labels, _, _, cluster_centers = cluster_kmeans(embeddings, 37)

# Crear un diccionario para contar el número de embeddings en cada clúster
clusters_count = {}
clusters_dict = {}

for label in final_cluster_labels:
    if label not in clusters_count:
        clusters_count[label] = 0
        clusters_dict[label] = []  # Inicializa la lista para este cluster
    clusters_count[label] += 1

# Diccionario para almacenar las etiquetas de los clústeres
cluster_labels = {}
document_labels = []  # Lista para almacenar la info del document_id, document_path, content_vector y etiqueta

for label, count in clusters_count.items():
    print(f'Clúster {label}: {count} embeddings')

    # Obtener los embeddings y el contenido en el clúster actual
    cluster_embeddings = embeddings[final_cluster_labels == label]
    cluster_ids = np.array(document_ids)[final_cluster_labels == label]  # Obtener document_id
    cluster_paths = np.array(document_paths)[final_cluster_labels == label]  # Obtener document_path
    cluster_texts = np.array(contents)[final_cluster_labels == label]  # Obtener contenidos

    # Calcular la distancia de cada embedding al centroide del clúster
    distances = cdist([cluster_centers[label]], cluster_embeddings, metric='euclidean')[0]

    # Crear un diccionario para almacenar los documentos por clúster
    clusters_dict[label] = []
    for doc_id, embedding, labels, path in zip(document_ids, embeddings, final_cluster_labels, document_paths):
        if labels not in clusters_dict:
            clusters_dict[labels] = []
        clusters_dict[labels].append((doc_id, embedding, path))
    
        # Generate a label for the cluster using GPT
    etiqueta = generate_cluster_label(cluster_texts)

    print(f'Etiqueta generada para el clúster {label}: {etiqueta} ')
        
        # Almacenar la etiqueta en el diccionario de clústeres
    cluster_labels[label] = etiqueta